In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')

from tqdm import tqdm
import aoc_utils as au


In [3]:
input_text = au.read_txt_file_lines()
input_text

['seeds: 929142010 467769747 2497466808 210166838 3768123711 33216796 1609270159 86969850 199555506 378609832 1840685500 314009711 1740069852 36868255 2161129344 170490105 2869967743 265455365 3984276455 31190888',
 '',
 'seed-to-soil map:',
 '3788621315 24578909 268976974',
 '3633843608 2672619957 154777707',
 '1562003446 2827397664 767899879',
 '2618130896 293555883 1015712712',
 '178572254 3595297543 462300746',
 '640873000 1553961386 921130446',
 '2373438105 1435027522 118933864',
 '2492371969 1309268595 125758927',
 '2329903325 2629085177 43534780',
 '24578909 2475091832 153993345',
 '',
 'soil-to-fertilizer map:',
 '3438795585 3489901885 246180709',
 '2207634271 2882741084 20404178',
 '2921657379 3093830975 5578539',
 '3684976294 494116589 47356386',
 '3924272359 3740035076 65483093',
 '501906244 2178419272 30310459',
 '1234683323 1988217660 166809869',
 '1901474422 3380297447 5929907',
 '1561485292 902794130 43191607',
 '3740035076 4064176085 35875061',
 '1756130945 2337296126 1

In [4]:
list_seeds = [int(x) for x in input_text[0].lstrip('seeds: ').split()]
list_maps = input_text[2:]


## plan part 1:
extract map values into dictionary 
then create if statements to do the mapping. 

In [5]:
dict_map_values = {}
for l in list_maps:
    if 'map' in l:
        # print(l)
        name_map = l.rstrip(' map:')
        dict_map_values[name_map] = []
    elif l == '':
        pass 
    else:
        nums = [int(x) for x in l.split()]
        assert len(nums) == 3
        tmp_dict = {}
        tmp_dict['dest'] = nums[0]
        tmp_dict['source'] = nums[1]
        tmp_dict['len'] = nums[2]
        dict_map_values[name_map].append(tmp_dict)

def transform_seed_to_location(seed, dict_map_values):
    dict_solution = {'seed': seed}
    curr_name = 'seed'
    curr_val = seed
    for map_name in dict_map_values.keys():
        map_name_split = map_name.split('-')
        assert len(map_name_split) == 3
        assert map_name_split[0] == curr_name 
        new_name = map_name_split[2]
        new_val = None
        for d in dict_map_values[map_name]:
            if curr_val >= d['source'] and curr_val < d['source'] + d['len']:
                new_val = d['dest'] + curr_val - d['source']
                break
        if new_val is None:
            new_val = curr_val 
        dict_solution[new_name] = new_val
        # print(f'found {new_name} = {new_val} from {curr_name} = {curr_val}')
        curr_name = new_name
        curr_val = new_val

    return dict_solution

def transform_seed_to_location_fast(seed, dict_map_values):
    curr_val = seed
    for map_name in dict_map_values.keys():
        for d in dict_map_values[map_name]:
            if curr_val >= d['source'] and curr_val < d['source'] + d['len']:
                new_val = d['dest'] + curr_val - d['source']
                break
        if new_val is None:
            new_val = curr_val 
        curr_val = new_val

    return curr_val

def list_all_locations(list_seeds, dict_map_values, find_min=True):

    dict_solutions = {}
    for seed in list_seeds:
        dict_solutions[seed] = transform_seed_to_location(seed, dict_map_values)['location']

    if find_min:
        min_val = min(dict_solutions.values())
        for k in dict_solutions.keys():
            if dict_solutions[k] == min_val:
                print(f'found min location {min_val} at seed {k}')
                break

    return dict_solutions

dict_solutions = list_all_locations(list_seeds, dict_map_values)
# dict_solutions

found min location 600279879 at seed 3984276455


## part 2
should be v easy now! :) 

(Or are the numbers too large to brute force?) - > yes .. 


In [6]:
list_seeds

[929142010,
 467769747,
 2497466808,
 210166838,
 3768123711,
 33216796,
 1609270159,
 86969850,
 199555506,
 378609832,
 1840685500,
 314009711,
 1740069852,
 36868255,
 2161129344,
 170490105,
 2869967743,
 265455365,
 3984276455,
 31190888]

In [7]:
%timeit transform_seed_to_location(list_seeds[0], dict_map_values)

8.18 µs ± 70.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [9]:
%timeit transform_seed_to_location_fast(list_seeds[0], dict_map_values)  # not fast enough

6.92 µs ± 56.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [15]:
len_seeds1 = len(list_seeds)
assert len_seeds1 % 2 == 0
arr_seed_ranges_approx = np.zeros((len_seeds1 // 2, 2), dtype=int)
arr_seed_ranges_precise = np.zeros((len_seeds1 // 2, 2), dtype=int)

for ii in range(len_seeds1 // 2):

    start_range_ind = int(ii * 2)
    len_range_ind = int(ii * 2 + 1)
    start_range = list_seeds[start_range_ind]
    len_range = list_seeds[len_range_ind]
    end_range = start_range + len_range - 1
    arr_seed_ranges_approx[ii, 0] = int(start_range / 1e7)
    arr_seed_ranges_approx[ii, 1] = int(end_range / 1e7)
    arr_seed_ranges_precise[ii, 0] = start_range
    arr_seed_ranges_precise[ii, 1] = end_range
    
arr_seed_ranges_approx = arr_seed_ranges_approx[arr_seed_ranges_approx[:, 0].argsort()]
arr_seed_ranges_precise = arr_seed_ranges_precise[arr_seed_ranges_precise[:, 0].argsort()]
arr_seed_ranges_approx

array([[ 19,  57],
       [ 92, 139],
       [160, 169],
       [174, 177],
       [184, 215],
       [216, 233],
       [249, 270],
       [286, 313],
       [376, 380],
       [398, 401]])

## Some thoughts

1. Brute-forcing all numbers doesnt work. 
2. Seems sensible to try to work with ranges. Eg backtrack from locations: how do ranges split up into humidity etc.? Difficulty with back-tracking is that I don't think the solution is isomorph in the sense that multiple humidity values could map to the same location value.. Therefore I think you should stick with forward passes. Can I work with ranges in forward pass? Not sure how to (easily code this)
3. Rogue idea: on the one hand, the mapping from seed to location is very non-linear. However, there is some local continuity. The sources/dest/ranges are very large (1e7-1e9), so there must be a lot of local continuity. Maybe approach it like a parameter sweep optimisation: first optimise every 1000th, then assume that the global minimum is close to the local minimum found there .. 

Update: tried **3.** and it worked! feels very hacky but ah well .. 

In [16]:
loc_min = np.inf
seed_min = None

## Try every 1000th seed:
for ii in range(len_seeds1 // 2):
    ## get seed range
    start_range_ind = int(ii * 2)
    len_range_ind = int(ii * 2 + 1)
    start_range = list_seeds[start_range_ind]
    len_range = list_seeds[len_range_ind]
    end_range = start_range + len_range - 1

    ## try every 1000th seed
    for seed in tqdm(range(start_range, end_range + 1, 1000)):        
        loc = transform_seed_to_location(seed, dict_map_values)['location']
        if loc < loc_min:
            loc_min = loc
            seed_min = seed
            print(f'Found new best seed {seed_min} with location {loc_min}')
print(f'final best: seed {seed_min} with location {loc_min}')

  5%|▌         | 23474/467770 [00:00<00:03, 118682.89it/s]

Found new best seed 929142010 with location 2928341086
Found new best seed 933492010 with location 2883781920


 17%|█▋        | 79081/467770 [00:00<00:02, 139281.02it/s]

Found new best seed 977138010 with location 1875261567
Found new best seed 987753010 with location 1553251742


 46%|████▌     | 213316/467770 [00:01<00:02, 108524.10it/s]

Found new best seed 1129825010 with location 23846622


 55%|█████▌    | 208263/378610 [00:01<00:01, 133412.28it/s]

Found new best seed 391178506 with location 20191348


100%|██████████| 31191/31191 [00:00<00:00, 157170.76it/s]

final best: seed 391178506 with location 20191348


Now evaluate every 100th of the entire seed range that contained the best previously:

In [17]:
loc_min = np.inf

for ii in range(len_seeds1 // 2):
    start_range_ind = int(ii * 2)
    len_range_ind = int(ii * 2 + 1)
    start_range = list_seeds[start_range_ind]
    len_range = list_seeds[len_range_ind]
    end_range = start_range + len_range - 1

    ## Only do the 4th range (which is the one with previous best seed 391178506)
    if ii == 4:
        print(f'range {ii}: {start_range} to {end_range}')
    else:
        continue

    ## try every 100th
    for seed in tqdm(range(start_range, end_range + 1, 100)):        
        loc = transform_seed_to_location(seed, dict_map_values)['location']
        if loc < loc_min:
            loc_min = loc
            seed_min = seed
            print(f'Found new best seed {seed_min} with location {loc_min}')
    break # don't do the rest after this range 

print(f'final best: seed {seed_min} with location {loc_min}')

range 4: 199555506 to 578165337


  1%|          | 19867/3786099 [00:00<00:37, 99779.71it/s]

Found new best seed 199555506 with location 2768526595


  3%|▎         | 101691/3786099 [00:00<00:28, 129195.07it/s]

Found new best seed 207005906 with location 1468108862


...

Found new best seed 243884106 with location 1228300982


 51%|█████     | 1937598/3786099 [00:13<00:14, 127602.20it/s]

Found new best seed 391178306 with location 20191148


100%|██████████| 3786099/3786099 [00:29<00:00, 129644.51it/s]

final best: seed 391178306 with location 20191148


Found almost the same seed again! only 200 different. Now try every seed around 391178506 (milliion seeds either side)

In [18]:
for seed in tqdm(range(391178506 - 1000000, 391178506 + 1000000)):
    loc = transform_seed_to_location(seed, dict_map_values)['location']
    if loc < loc_min:
        loc_min = loc
        seed_min = seed
        print(f'Found new best seed {seed_min} with location {loc_min}')

 51%|█████     | 1023291/2000000 [00:07<00:07, 126618.23it/s]

Found new best seed 391178260 with location 20191102


100%|██████████| 2000000/2000000 [00:15<00:00, 125185.56it/s]


And that's it! 